<a href="https://colab.research.google.com/github/tjtmddnjswkd/capstone/blob/master/api_crawling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install xmltodict

In [2]:
import pandas as pd
import re
import sys
import requests
import json
import pprint
from bs4 import BeautifulSoup
from urllib.request import urlopen
import xmltodict
from tqdm import tqdm
import random
import pprint

In [3]:
url = 'http://www.law.go.kr/DRF/lawSearch.do?OC=seokhee051&target=elaw&type=XML'
# url 불러오기
response = requests.get(url)
# XML -> 딕셔너리 변환후 파싱
contents = response.text
dict_contents = xmltodict.parse(contents)
totalCnt = int(dict_contents['LawSearch']['totalCnt'])

In [4]:
totalCnt

2610

In [ ]:
rows = []
not_match_jo = []
changed_num = 0
changed = []
eng_cnt = 0
kor_cnt = 0
total_page = int(totalCnt/20)+1
for p in tqdm(range(120,total_page)):
  url = 'http://www.law.go.kr/DRF/lawSearch.do?OC=seokhee051&target=elaw&type=XML' \
        + '&page=' + str(p)
  # url 불러오기
  response = requests.get(url)
  # XML -> 딕셔너리 변환후 파싱
  contents = response.text
  dict_contents = xmltodict.parse(contents)
  try:
    for content in dict_contents['LawSearch']['law']:
      name_kor = content['법령명한글']
      name_eng = content['법령명영문']
      mst = content['법령일련번호']

      eng_url = 'https://www.law.go.kr/DRF/lawService.do?OC=seokhee051&target=elaw' \
                + '&MST=' + str(mst)
                # + '&ID=' + str(법령ID)
      eng_response = requests.get(eng_url)
      eng_contents = eng_response.text
      eng_dict_contents = xmltodict.parse(eng_contents)

      kor_url = 'https://www.law.go.kr/DRF/lawService.do?OC=seokhee051&target=law' \
                + '&MST=' + str(mst)
                # + '&ID=' + str(법령ID)
      kor_response = requests.get(kor_url)
      kor_contents = kor_response.text
      kor_dict_contents = xmltodict.parse(kor_contents)

      # # 영어조문 개수와 한글 조문 개수 맞지 않다면 건너뛰기
      # if len(eng_dict_contents['Law']['JoSection']['Jo']) != len(kor_dict_contents['법령']['조문']['조문단위']):
      #     not_match_jo += 1
      #     print(f'영한 조문 개수 맞지 않는 법령 개수', not_match_jo, kor_dict_contents['법령']['기본정보']['법령명_한글'])
      #     continue
      # eng_cnt += len(eng_dict_contents['Law']['JoSection']['Jo'])
      # kor_cnt += len(kor_dict_contents['법령']['조문']['조문단위'])
      min_num_jo = min(len(eng_dict_contents['Law']['JoSection']['Jo']), len(kor_dict_contents['법령']['조문']['조문단위']))
      temp_cnt = 0
      print('법령명:', name_kor, ', 조문 개수:', min_num_jo, ', 페이지:', p, ', 몇번째?:', content['@id'])
      try:
        # for eng_jo, kor_jo in zip(eng_dict_contents['Law']['JoSection']['Jo'],kor_dict_contents['법령']['조문']['조문단위']):
        for ind in range(min_num_jo):
          eng_jo, kor_jo = eng_dict_contents['Law']['JoSection']['Jo'][ind], kor_dict_contents['법령']['조문']['조문단위'][ind]

          if kor_jo['조문여부']!='조문':
            continue
          # if kor_jo.get('조문가지번호'):
          #   continue

          kor_total = 0
          if kor_jo.get('항'): # 항내용이 있으면 list 바로 호내용이면 dict 타입이네
            if type(kor_jo['항']) == dict:
              kor_total += len(kor_jo['항']['호'])
              for ho in kor_jo['항']['호']:
                if ho.get('목'):
                  kor_total += len(ho['목'])
              kor_total += 1 # 호부터 시작하면 영어에서는 또 \n이 하나 늘어남
            else:
              kor_total += len(kor_jo['항'])
              for hang in kor_jo['항']:
                if hang.get('호'):
                  kor_total += len(hang['호'])
                  for ho in hang['호']:
                    if ho.get('목'):
                      kor_total += len(ho['목'])
          else:
            kor_total = 1

          if kor_jo.get('조문참고자료'):
            if '\n\t\t\n\t\t' in kor_jo['조문참고자료']:
              kor_total += 2
              kor_jo['조문참고자료'] = kor_jo['조문참고자료'].replace('\n\t\t\n\t\t', '\n')
            else:
              kor_total += 1

          if (eng_jo['joCts'].count('\n')+1) != kor_total:
            not_match_jo.append([name_kor, kor_jo['조문제목']])
            continue

          if eng_jo['joYn']!= 'Y':
            changed_num += 1
            changed.append([eng_jo, kor_jo])
            print('안바뀐 법령 개수:', changed_num)
            continue
          # if eng_jo['joBrNo']!='00':
          #   continue

          eng_text = eng_jo['joCts']

          jo_txt = ''
          hang_txt = ''
          ho_txt = ''
          mok_txt = ''

          jo_txt += kor_jo['조문내용']
          if kor_jo.get('항'): # 항내용이 있으면 list 바로 호내용이면 dict 타입이네
            if type(kor_jo['항']) == dict: # 바로 호내용일 때
              for ho in kor_jo['항']['호']:
                ho_txt += '\n' + ho['호내용']
                if ho.get('목'):
                  for mok in ho['목']:
                    mok_txt += '\n' + mok['목내용']
                  ho_txt += mok_txt
              hang_txt += ho_txt
            else:
              for hang in kor_jo['항']:
                if hang_txt == '':
                  hang_txt += hang['항내용'] # 영어의 경우 맨 첫항과 조문 내용 사이에 \n으로 분리가 안돼있음
                else:
                  hang_txt += '\n' + hang['항내용']

                if hang.get('호'):
                  for ho in hang['호']:
                    ho_txt += '\n' + ho['호내용']
                    if ho.get('목'):
                      for mok in ho['목']:
                        mok_txt += '\n' + mok['목내용']
                      ho_txt += mok_txt
                  hang_txt += ho_txt
              jo_txt += hang_txt
          if kor_jo.get('조문참고자료'):
            eng_text = '\n'.join([i.strip() for i in eng_text.split('\n')[:-1]]) # 조문참고자료 부분 삭제하기
          kor_text = jo_txt
          rows.append({'name_kor':name_kor, 'name_eng':name_eng, 'MST':mst, 'eng':eng_text, 'kor':kor_text})
      except Exception as e:
        print('eng_kor_contents 예외 발생:', e)
  except Exception as e:
    print('dict_contents 예외 발생:', e)
print(len(not_match_jo))
df = pd.DataFrame(rows)

In [ ]:
df.to_csv('/content/drive/MyDrive/KB/KB_trans/120-.csv', index=False)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [143]:
url = 'http://www.law.go.kr/DRF/lawSearch.do?OC=seokhee051&target=elaw&type=XML' \
        + '&page=1'
# url 불러오기
response = requests.get(url)
# XML -> 딕셔너리 변환후 파싱
contents = response.text
dict_contents = xmltodict.parse(contents)

content = dict_contents['LawSearch']['law'][1]

name_kor = content['법령명한글']
name_eng = content['법령명영문']
mst = content['법령일련번호']
print(name_kor)
eng_url = 'https://www.law.go.kr/DRF/lawService.do?OC=seokhee051&target=elaw' \
          + '&MST=' + str(mst)
          # + '&ID=' + str(법령ID)
eng_response = requests.get(eng_url)
eng_contents = eng_response.text
eng_dict_contents = xmltodict.parse(eng_contents)

kor_url = 'https://www.law.go.kr/DRF/lawService.do?OC=seokhee051&target=law' \
          + '&MST=' + str(mst)
          # + '&ID=' + str(법령ID)
kor_response = requests.get(kor_url)
kor_contents = kor_response.text
kor_dict_contents = xmltodict.parse(kor_contents)

119구조ㆍ구급에 관한 법률 시행령


In [ ]:
count = 0
for temp in kor_dict_contents['법령']['조문']['조문단위']:
    if temp.get('조문제목'):
        if temp['조문제목'] == '기념화폐의 판매':
            print(count)
    count += 1

17


In [152]:
eng_dict_contents['Law']['JoSection']['Jo'][7]

{'@No': '8',
 'chapNo': '00000300000000000000',
 'joNo': '0005',
 'joBrNo': '00',
 'joYn': 'Y',
 'joTtl': 'Organization and Operation of 119 Rescue Squads',
 'joCts': 'Article 5 (Organization and Operation of 119 Rescue Squads) (1) Any 119 rescue squads under Article 8 (1) of the Act (hereinafter referred to as “rescue squad”) shall be organized and operated by classifications as follows: <Amended on Jul. 7, 2014; Jul. 14, 2014; Nov. 19, 2014; Oct. 25, 2016; Jul. 26, 2017>\n  1. General rescue squad: At least one squad shall be established in each fire station as prescribed by Municipal Rule of a City/Do, but where a Si/Gun/Gu (a Gu refers to an autonomous Gu; hereinafter the same apply) does not have a fire station, the squad may be established in an 119 safety center located in the center of the relevant Si/Gun/Gu;\n  2. Special rescue squad: A squad shall be established in a fire station having jurisdiction over an area classified in the following items as prescribed by Municipal Ru

In [151]:
kor_dict_contents['법령']['조문']['조문단위'][7]

{'@조문키': '0005001',
 '조문번호': '5',
 '조문여부': '조문',
 '조문제목': '119구조대의 편성과 운영',
 '조문시행일자': '20210121',
 '조문이동이전': '000000',
 '조문이동이후': '000000',
 '조문변경여부': 'N',
 '조문내용': '제5조(119구조대의 편성과 운영)',
 '항': [{'항번호': '①',
   '항내용': '① 법 제8조제1항에 따른 119구조대(이하 "구조대"라 한다)는 다음 각 호의 구분에 따라 편성·운영한다. <개정 2014.7.7, 2014.7.14, 2014.11.19, 2016.10.25, 2017.7.26>',
   '호': [{'호번호': '1.',
     '호내용': '1. 일반구조대: 시·도의 규칙으로 정하는 바에 따라 소방서마다 1개 대(隊) 이상 설치하되, 소방서가 없는 시·군·구(자치구를 말한다. 이하 같다)의 경우에는 해당 시·군·구 지역의 중심지에 있는 119안전센터에 설치할 수 있다.'},
    {'호번호': '2.',
     '호내용': '2. 특수구조대: 소방대상물, 지역 특성, 재난 발생 유형 및 빈도 등을 고려하여 시·도의 규칙으로 정하는 바에 따라 다음 각 목의 구분에 따른 지역을 관할하는 소방서에 다음 각 목의 구분에 따라 설치한다. 다만, 라목에 따른 고속국도구조대는 제3호에 따라 설치되는 직할구조대에 설치할 수 있다.',
     '목': [{'목번호': '가.', '목내용': '가. 화학구조대: 화학공장이 밀집한 지역'},
      {'목번호': '나.', '목내용': '나. 수난구조대: 「내수면어업법」 제2조제1호에 따른 내수면지역'},
      {'목번호': '다.', '목내용': '다. 산악구조대: 「자연공원법」 제2조제1호에 따른 자연공원 등 산악지역'},
      {'목번호': '라.',
       '목내용': '라. 고속국도구조대: 「도로법」 제10조제1호에 따른 고속국도(이하 "고속국도"라 한다)'},
   

In [ ]:
# 영한 조문 개수 맞지 않는 법령 개수 2 간접투자자산 운용업법 시행령 17
# 영한 조문 개수 맞지 않는 법령 개수 3 간척지의 농어업적 이용 및 관리에 관한 법률 9
# 영한 조문 개수 맞지 않는 법령 개수 4 개발이익 환수에 관한 법률 14
# 영한 조문 개수 맞지 않는 법령 개수 5 개별소비세법 18

In [ ]:
kor_jo = kor_dict_contents['법령']['조문']['조문단위'][43]
eng_jo = eng_dict_contents['Law']['JoSection']['Jo'][43]
kor_total = 0
if kor_jo.get('항'): # 항내용이 있으면 list 바로 호내용이면 dict 타입이네
  if type(kor_jo['항']) == dict:
    kor_total += len(kor_jo['항']['호'])
    for ho in kor_jo['항']['호']:
      if ho.get('목'):
        kor_total += len(ho['목'])
    kor_total += 1
  else:
    kor_total += len(kor_jo['항'])
    for hang in kor_jo['항']:
      if hang.get('호'):
        kor_total += len(hang['호'])
        for ho in hang['호']:
          if ho.get('목'):
            kor_total += len(ho['목'])
else:
  kor_total = 1
print(kor_total)

if kor_jo.get('조문참고자료'):
  kor_total += (1 + kor_jo['조문참고자료'].count('\n'))
print(kor_total)
if (eng_jo['joCts'].count('\n')+1) != kor_total:
  not_match_jo += 1
  print(not_match_jo)

6
9
4
